In [1]:
import os

In [2]:
%pwd

'd:\\SEM - 4\\Projects\\End-to-End-ML-Project-with-MLflow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\SEM - 4\\Projects\\End-to-End-ML-Project-with-MLflow'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
import sys
import os

# Add 'src' folder to Python path
project_root = r"D:\SEM - 4\Projects\End-to-End-ML-Project-with-MLflow\src"
sys.path.append(project_root)


In [7]:
from MLflowProject.constants import *
from MLflowProject.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [10]:
import os
import urllib.request as request
import zipfile
from MLflowProject import logger
from MLflowProject.utils.common import get_size

In [11]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

In [17]:
print(data_ingestion_config.source_URL)


https://github.com/gargig13/RepoPrac/raw/refs/heads/main/winequality-data.zip


In [18]:
from urllib.error import HTTPError
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-08-15 16:56:34,633: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-15 16:56:34,637: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-15 16:56:34,640: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-08-15 16:56:34,643: INFO: common: created directory at: artifacts]
[2025-08-15 16:56:34,647: INFO: common: created directory at: artifacts/data_ingestion]


[2025-08-15 16:56:41,433: INFO: 2366278714: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: D310:7B707:96566:1A4AEF:689F1971
Accept-Ranges: bytes
Date: Fri, 15 Aug 2025 11:26:41 GMT
Via: 1.1 varnish
X-Served-By: cache-hyd1100030-HYD
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1755257201.179424,VS0,VE238
Vary: Authorization,Accept-Encoding
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: a4f74c87887ada8d31165248af19e2ae050a930f
Expires: Fri, 15 Aug 2025 11:31:41 GMT
Source-Age: 0

]
